In [115]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from sklearn.preprocessing import MinMaxScaler
import os

from tensorflow.keras.layers import Conv2D,InputLayer, Input, concatenate ,RepeatVector ,Reshape ,UpSampling2D

from tensorflow.keras.models import Model, load_model, Sequential



from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint



In [ ]:
# data from https://www.floydhub.com/emilwallner/datasets/colornet
# download data an put it in a folder in the root called "data"
items = []
num = 0
for file in os.listdir("./data/images/Train/"):
    if (num < 250):

        img_array = img_to_array(load_img("./data/images/Train/" + file))
        items.append(img_array)
    num += 1
items = np.array(items)
X_train = 1.0/255 * items

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, decode_predictions, preprocess_input
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('/kaggle/input/dataset/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.get_default_graph()

In [ ]:
#To generate embeddings of 1000*1 by passing input images through InceptionResNetV2
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed
import tensorflow as tf

In [5]:
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True
)


def image_train_datagen(batch_size):
    for batch_imgs in train_datagen.flow(X_train, batch_size=batch_size):

        grayscale_for_embeding = gray2rgb(rgb2gray(batch_imgs))
    
        embeding = embed_inception_prediction(grayscale_for_embeding)

        lab_train = rgb2lab(batch_imgs)
        l_batch = lab_train[:, :, :, 0]
        l_batch = l_batch.reshape(l_batch.shape + (1,))

        ab_batch = lab_train[:, :, :, 1:] / 128
       
        yield([l_batch, embeding], ab_batch)



X_train (20000, 224, 224, 1)
X_test (5000, 224, 224, 1)
y_train (20000, 224, 224, 2)
y_test (5000, 224, 224, 2)
0
255
20
226


In [ ]:

from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, concatenate, Activation, Dense, Dropout, Flatten, RepeatVector
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
embedings_input = Input(shape=(1000, ))

# using kears functional API to pass the outputs to each layer
encoder_input = Input(shape=(256, 256, 1,))
encoder1 = Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(encoder_input)
encoder2 = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder1)
encoder3 = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(encoder2)
encoder4 = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder3)
encoder5 = Conv2D(256, (3, 3), activation='relu', padding='same', strides=(2, 2))(encoder4)
encoder6 = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder5)
encoder7 = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder6)
encoder_output = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder7)

fusion1 = RepeatVector(32 * 32)(embedings_input)
fusion2 = Reshape(([32, 32, 1000]))(fusion1)
fusion3 = concatenate([encoder_output, fusion2], axis=3)
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion3)

decoder1 = Conv2D(128, (3, 3), activation='relu', padding='same')(fusion_output)
decoder2 = UpSampling2D((2, 2))(decoder1)
decoder3 = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder2)
decoder4 = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder3)
decoder5 = UpSampling2D((2, 2))(decoder4)
decoder6 = Conv2D(32, (3, 3), activation='relu', padding='same')(decoder5)
decoder7 = Conv2D(16, (3, 3), activation='relu', padding='same')(decoder6)
decoder8 = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder7)
final_decoder_output = UpSampling2D((2, 2))(decoder8)

myModel = Model(inputs=[encoder_input, embedings_input], outputs=final_decoder_output)

In [ ]:
model_path = './chroma_model.h5'

make_checkpoint = ModelCheckpoint(model_path,
                            monitor = "val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose = 1)
early_stoping = EarlyStopping(monitor='val_loss', mode='min', verbose = 1)

myModel.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

In [ ]:
batch_size_number = 10 # batch
num_epochs = 1
# total number of imgs / batch size
steps_per_epoch_num = 929
history = myModel.fit_generator(image_train_datagen(batch_size_number), epochs = num_epochs, steps_per_epoch=steps_per_epoch_num, verbose = 2, callbacks = [make_checkpoint,early_stoping])